# Cincinnati Model Architecture Part 2


In this notebook, the model architecture from the NYC taxi data Part 3 is applied to the Cincinnati Vehicle DataSet.
Part 3 analyzed an architecture that only predicts based on a location sequence input.
The results have to be compared with Part 1 in order to see if the conclusions drawn from Part 1 to 7 can also be applied to the Cincinnati Vehiclel DataSet.


In [1]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow.keras import layers
import import_ipynb

In [2]:
from model_helper import ModelHelper

importing Jupyter notebook from model_helper.ipynb


# Dataset

In [3]:
df = pd.read_csv("./cincinnati/cincinatti_zones_153.csv")
df.head(10)

,location_id,vehicle_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,4,153,False,-0.725424,-0.688302,0.743145,0.669131,0.5,0.866025,0.0,1.0
1,10,153,False,-0.619379,-0.785092,0.743145,0.669131,0.5,0.866025,0.0,1.0
2,4,153,False,0.378379,-0.925651,0.743145,0.669131,0.5,0.866025,0.0,1.0
3,4,153,False,0.635798,-0.771856,0.743145,0.669131,0.5,0.866025,0.0,1.0
4,11,153,False,-0.492424,-0.870356,0.743145,0.669131,0.5,0.866025,0.0,1.0
5,4,153,False,0.476903,-0.878956,0.743145,0.669131,0.5,0.866025,0.0,1.0
6,18,153,False,-0.697999,-0.716099,0.743145,0.669131,0.5,0.866025,0.0,1.0
7,10,153,False,0.121725,-0.992564,0.743145,0.669131,0.5,0.866025,0.0,1.0
8,10,153,False,0.004363,-0.999990,0.743145,0.669131,0.5,0.866025,0.0,1.0
9,13,153,False,0.618923,-0.785452,0.743145,0.669131,0.5,0.866025,0.0,1.0


In [4]:
df.drop(columns=['vehicle_id'], axis='columns', inplace=True)
df

,location_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,4,False,-0.725424,-0.688302,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
1,10,False,-0.619379,-0.785092,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
2,4,False,0.378379,-0.925651,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
3,4,False,0.635798,-0.771856,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
4,11,False,-0.492424,-0.870356,0.743145,0.669131,0.500000,0.866025,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
165636,0,False,-0.108939,-0.994048,-0.587785,0.809017,-0.500000,-0.866025,0.781831,0.623490
165637,0,False,-0.083026,-0.996547,0.743145,0.669131,-0.866025,-0.500000,0.974928,-0.222521
165638,1,True,0.321302,-0.946977,0.207912,0.978148,-0.500000,-0.866025,-0.974928,-0.222521
165639,1,False,-0.933554,0.358436,0.743145,0.669131,-0.866025,-0.500000,0.974928,-0.222521


In [5]:
# the number of different locations defines the vocabulary size
locations = df.location_id
vocab_size = locations.nunique()
print('vocabulary size:', vocab_size)

vocabulary size: 47


In [6]:
df['location_id'] = df.groupby('location_id', sort=False).ngroup()
df.head(10)

,location_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,0,False,-0.725424,-0.688302,0.743145,0.669131,0.5,0.866025,0.0,1.0
1,1,False,-0.619379,-0.785092,0.743145,0.669131,0.5,0.866025,0.0,1.0
2,0,False,0.378379,-0.925651,0.743145,0.669131,0.5,0.866025,0.0,1.0
3,0,False,0.635798,-0.771856,0.743145,0.669131,0.5,0.866025,0.0,1.0
4,2,False,-0.492424,-0.870356,0.743145,0.669131,0.5,0.866025,0.0,1.0
5,0,False,0.476903,-0.878956,0.743145,0.669131,0.5,0.866025,0.0,1.0
6,3,False,-0.697999,-0.716099,0.743145,0.669131,0.5,0.866025,0.0,1.0
7,1,False,0.121725,-0.992564,0.743145,0.669131,0.5,0.866025,0.0,1.0
8,1,False,0.004363,-0.999990,0.743145,0.669131,0.5,0.866025,0.0,1.0
9,4,False,0.618923,-0.785452,0.743145,0.669131,0.5,0.866025,0.0,1.0


In [7]:
mh = ModelHelper(df, 17)

In [8]:
mh.set_target_column_name('location_id')
mh.set_vocab_size(vocab_size)
numerical_column_names = ['start_hour_sin', 'start_hour_cos', 'weekend', 'week_day_sin', 'week_day_cos']
column_names = ['location_id'] + numerical_column_names
mh.set_column_names(column_names)
mh.set_numerical_column_names(numerical_column_names)

In [9]:
# Call the function
mh.train_val_test_split()
print(len(mh.df_train), 'train examples')
print(len(mh.df_val), 'validation examples')
print(len(mh.df_test), 'test examples')

106009 train examples
26503 validation examples
33129 test examples


In [10]:
mh.split_data()
print(len(mh.list_test))
mh.list_test[0]

1949


,location_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
132512,34,False,-0.915429,-0.402480,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132513,8,False,-0.999767,-0.021597,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132514,15,False,0.186524,-0.982450,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132515,4,False,-0.694815,-0.719188,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132516,4,False,-0.865989,0.500063,0.994522,-0.104528,-0.5,-0.866025,0.433884,-0.900969
132517,12,False,0.216298,-0.976327,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132518,10,False,-0.026468,-0.999650,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132519,18,False,-0.978986,0.203927,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969
132520,8,False,-0.918734,0.394877,0.994522,-0.104528,-0.5,-0.866025,0.433884,-0.900969
132521,8,False,-0.995792,0.091646,0.951057,-0.309017,-0.5,-0.866025,-0.433884,-0.900969


In [11]:
mh.set_batch_size(16)
mh.create_batch_dataset_cin()
mh.test_dataset

<BatchDataset shapes: ({start_place: (16, 16), start_hour_sin: (16, 16), start_hour_cos: (16, 16), weekend: (16, 16), week_day_sin: (16, 16), week_day_cos: (16, 16), end_hour_sin: (16,), end_hour_cos: (16,), end_weekend: (16,), end_week_day_sin: (16,), end_week_day_cos: (16,)}, (16,)), types: ({start_place: tf.int64, start_hour_sin: tf.float64, start_hour_cos: tf.float64, weekend: tf.bool, week_day_sin: tf.float64, week_day_cos: tf.float64, end_hour_sin: tf.float64, end_hour_cos: tf.float64, end_weekend: tf.bool, end_week_day_sin: tf.float64, end_week_day_cos: tf.float64}, tf.int64)>

In [12]:
def sparse_f(input_dense):
  zero = tf.constant(0, dtype=tf.float32)
  indices = tf.where(tf.not_equal(input_dense, zero))
  values = tf.gather_nd(input_dense, indices)
  sparse = tf.SparseTensor(indices, values,  tf.cast(tf.shape(input_dense), dtype=tf.int64))
  return sparse

In [13]:
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units1 = 256
rnn_units2 = 128

# Create a model
def create_model():
  N = mh.total_window_length
  batch_size = mh.batch_size
  number_of_places = mh.vocab_size

	# Shortcut to the layers package
  l = tf.keras.layers

   # Declare the dictionary for the places sequence as before
  sequence_input = {
      'start_place': tf.keras.Input((N-1,), batch_size=batch_size, dtype=tf.dtypes.int32, name='start_place') # add batch_size=batch_size in case of stateful GRU
  }

  # Handling the categorical feature sequence using one-hot
  places_one_hot = feature_column.sequence_categorical_column_with_vocabulary_list(
      'start_place', [i for i in range(number_of_places)])

  # Embed the one-hot encoding
  places_embed = feature_column.embedding_column(places_one_hot, embedding_dim)

  # With an input sequence we can't use the DenseFeature layer, we need to use the SequenceFeatures
  sequence_features, sequence_length = tf.keras.experimental.SequenceFeatures(places_embed)(sequence_input)

  sequence_features = tf.ensure_shape(sequence_features, (batch_size, N-1, sequence_features.shape[2]))

  # Rnn
  recurrent = l.GRU(rnn_units1,
                        batch_size=batch_size, #in case of stateful
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(sequence_features)

  recurrent_2 = l.GRU(rnn_units2,
                        batch_size=batch_size, #in case of stateful
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(recurrent)

	# Last layer with an output for each places
  dense_1 = layers.Dense(number_of_places)(recurrent_2)

	# Softmax output layer
  output = l.Softmax()(dense_1)

	# To return the Model, we need to define its inputs and outputs
	# In out case, we need to list all the input layers we have defined
  inputs = list(sequence_input.values())

	# Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

In [14]:
# Get the model and compile it
mh.assign_model(create_model())
mh.compile_model()

In [15]:
mh.model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
start_place (InputLayer)     [(16, 16)]                0         
_________________________________________________________________
sequence_features (SequenceF ((None, None, 256), (None 12032     
_________________________________________________________________
tf_op_layer_EnsureShape (Ten [(16, 16, 256)]           0         
_________________________________________________________________
gru (GRU)                    (16, 16, 256)             394752    
_________________________________________________________________
gru_1 (GRU)                  (16, 128)                 148224    
_________________________________________________________________
dense (Dense)                (16, 47)                  6063      
_________________________________________________________________
softmax (Softmax)            (16, 47)                 

In [16]:
mh.set_num_epochs(10)
mh.fit_model()

Epoch 1/10


C:\Users\WoodPecker\Documents\Privat\HTW-Master\Sem3\PA\HumanMobilityPredictionMA\venv\lib\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['start_hour_sin', 'start_hour_cos', 'weekend', 'week_day_sin', 'week_day_cos', 'end_hour_sin', 'end_hour_cos', 'end_weekend', 'end_week_day_sin', 'end_week_day_cos'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


389/389 [==============================] - 5s 14ms/step - loss: 2.3035 - sparse_categorical_accuracy: 0.2762 - val_loss: 2.4318 - val_sparse_categorical_accuracy: 0.2242
Epoch 2/10
389/389 [==============================] - 5s 13ms/step - loss: 2.0321 - sparse_categorical_accuracy: 0.2942 - val_loss: 2.3087 - val_sparse_categorical_accuracy: 0.2532
Epoch 3/10
389/389 [==============================] - 5s 13ms/step - loss: 1.9515 - sparse_categorical_accuracy: 0.3061 - val_loss: 2.3193 - val_sparse_categorical_accuracy: 0.2693
Epoch 4/10
389/389 [==============================] - 5s 13ms/step - loss: 1.9073 - sparse_categorical_accuracy: 0.3088 - val_loss: 2.2862 - val_sparse_categorical_accuracy: 0.2745
Epoch 5/10
389/389 [==============================] - 5s 13ms/step - loss: 1.8630 - sparse_categorical_accuracy: 0.3139 - val_loss: 2.2716 - val_sparse_categorical_accuracy: 0.2771
Epoch 6/10
389/389 [==============================] - 5s 14ms/step - loss: 1.8190 - sparse_categorical_acc

# Evaluation

In [17]:
mh.evaluate_model()

121/121 [==============================] - 0s 4ms/step - loss: 2.3782 - sparse_categorical_accuracy: 0.2056


In [18]:
mh.model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
start_place (InputLayer)     [(16, 16)]                0         
_________________________________________________________________
sequence_features (SequenceF ((None, None, 256), (None 12032     
_________________________________________________________________
tf_op_layer_EnsureShape (Ten [(16, 16, 256)]           0         
_________________________________________________________________
gru (GRU)                    (16, 16, 256)             394752    
_________________________________________________________________
gru_1 (GRU)                  (16, 128)                 148224    
_________________________________________________________________
dense (Dense)                (16, 47)                  6063      
_________________________________________________________________
softmax (Softmax)            (16, 47)                 

In [19]:
mh.print_test_prediction_info()

logits
Shape :  (1936, 47)
Example [0] :  [1.47600636e-01 2.56634294e-03 1.22410869e-02 1.25272293e-02
 2.65351593e-01 1.22651260e-03 1.40217384e-02 4.02487768e-03
 3.10201049e-01 1.09797865e-02 1.32906185e-02 3.37821036e-03
 4.45496803e-03 4.22547804e-03 1.02020265e-03 1.65972039e-02
 5.16976579e-04 6.40415703e-04 1.15589947e-01 1.71479601e-02
 1.81289983e-03 2.23707547e-03 2.68939875e-05 6.42791041e-04
 3.93882859e-03 5.33878291e-03 8.18641740e-04 1.06184445e-04
 1.53154205e-03 4.68449900e-03 3.70926340e-03 1.63619639e-03
 8.23580995e-05 1.74279045e-03 1.85229128e-05 7.57618982e-04
 5.37505723e-04 2.75661144e-03 1.87545409e-03 4.53404570e-03
 1.07015111e-03 3.84218620e-05 1.72910665e-03 6.89387845e-04
 2.80754721e-05 3.11088297e-05 2.22099079e-05]
predictions
Shape :  (1936, 47)
Example [0] :  tf.Tensor(
[0.02409106 0.02083859 0.02104117 0.02104719 0.02710157 0.02081068
 0.02107867 0.020869   0.02834473 0.02101465 0.02106327 0.02085551
 0.02087798 0.02087319 0.02080639 0.02113303 0.0

The assumption proved correct.
The usage of time components as input features improves the prediction accuracy by almost 8%.
This shows us that the network could learn more difficult patterns for the public service vehicles in contrary to the taxis or human checkins.
Here, the network was shown to not only predict the next location as the last location of the client.